In [1]:
import os
import pandas as pd
import polars as pl

In [2]:
def find_path(filename):
    path = "./"

    filepath = ""
    for root,d_names,f_names in os.walk(path):
        if "users.csv" in f_names: break

    filepath = "".join([root, "/", filename])
    return filepath


## Pandas

In [3]:
t = pd.read_csv(find_path("transactions.csv"))
u = pd.read_csv(find_path("users.csv"))

# join and filter
f = pd.merge(t, u, on="user_id", suffixes=('_trxn', '_user'))
f = f.loc[(f.is_blocked_user == False) & (f.is_blocked_trxn == False)]


# group and agregate
res = (
    f
    .groupby("transaction_category_id", as_index=False)
    .agg(
        {
            'transaction_amount': "sum",
            'user_id':            "nunique"
        }
    )
    .rename(columns=
        {
            'transaction_amount': 'sum_amount',
            'user_id':            'num_users'
        }
    )
    .sort_values(by=['sum_amount'], ascending=False)
)

In [4]:
res

,transaction_category_id,sum_amount,num_users
2,2,5798.38,752
7,7,1305.54,63
8,8,-44.61,22
1,1,-732.80,734
6,6,-1975.81,200
5,5,-5241.47,408
0,0,-12952.93,538
3,3,-14186.12,693
4,4,-17524.64,596


## Polars

In [5]:
# read data
t = pl.read_csv(find_path("transactions.csv"))
u = pl.read_csv(find_path("users.csv"))


# join, filter, group and agregate
res = (
    t
    .join(u, on="user_id", suffix="_user")
    .filter((pl.col("is_blocked") == False) & (pl.col("is_blocked_user") == False))

    .groupby("transaction_category_id")
    .agg(
        [
            pl.sum     ("transaction_amount").alias("sum_amount"),
            pl.n_unique("user_id")           .alias("num_users"),
        ]
    )
    .sort("sum_amount", reverse=True)
)

In [6]:
res

transaction_category_id,sum_amount,num_users
i64,f64,u32
2,5798.38,752
7,1305.54,63
8,-44.61,22
1,-732.8,734
6,-1975.81,200
5,-5241.47,408
0,-12952.93,538
3,-14186.12,693
4,-17524.64,596
